In [1]:
import pandas as pd
import math
import random
import torch
import yaml
import os
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel





In [2]:

config_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), '..')), "config/main.yaml")
with open(config_path, "r") as file:
    config = yaml.safe_load(file)
DATASET_PATH =config["DATASET_PATH"]

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")


2025-01-13 16:08:26.994171: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-13 16:08:27.034391: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-13 16:08:27.729831: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [43]:
class Dataloader:
    def __init__(self, path_file, preprocessor):

        self.data =  pd.read_csv(path_file)
        self.preprocessor = preprocessor
        self.n_data_points = len(self.data)
    
    def __len__(self):
        return self.n_data_points
    
    def __getitem__(self, idx):

        row = self.data.iloc[idx]
        image = Image.open(row["images"]).convert("RGB")
        text = row["text"]
        pixel_values = self.preprocessor(image, return_tensors="pt").pixel_values
        text = self.preprocessor.tokenizer(text, return_tensors="pt").input_ids

        return pixel_values, text

    def __iter__(self):
        self.idx = 0
        return self  

    def __next__(self):
        if self.idx >= self.n_data_points:
            self.idx = 0
            raise StopIteration
        self.idx += 1
        return self[self.idx] ## calling the 
    
    def collate_fn(self, batch):
        print(batch[0][1].shape,batch[5][1].shape )
        images  = torch.cat([data[0] for data in batch ], dim=0)
        max_length = max(len(seq) for seq in batch[1])
        # padded_input_ids = torch.tensor([data[1].tolist() + [self.preprocessor.tokenizer.pad_token_id] * (max_length - len(data)) for data in batch])
        padded_input_ids = torch.nn.utils.rnn.pad_sequence([data[1].squeeze(0) for data in batch],batch_first=True, padding_value=self.preprocessor.tokenizer.pad_token_id)
        return images, padded_input_ids
        


In [48]:
data = Dataloader(DATASET_PATH, preprocessor=processor)
train_loader = torch.utils.data.DataLoader(
    dataset     = data,
    batch_size  = config['BATCH_SIZE'],
    shuffle     = True,
    collate_fn= data.collate_fn
    )

for batch in train_loader:
    print(batch[0].shape, processor.tokenizer.batch_decode(batch[1], skip_special_tokens=True))
    break

torch.Size([1, 3]) torch.Size([1, 3])
torch.Size([8, 3, 384, 384]) ['14', '27/04/2000', '02/11/1988', 'Bisrat Getachew Tesfaye', '2', '22', 'Almaz@gmail.com', '-']
